# 概要
* ダイをきれいに要約させてみる
* 公式ドキュメントの通りtree_summarizeが要約最強である

# 参考
* tokenizerは最初に与えられたmax_chank内で文章を切り取り、separatorにて分割を行う。つまりmax_chankの処理が先でseparatorが後

* むしろchank_sizeは1行くらいのほうがいいかもしれない
* だいぶ良くなったけどまだ誤りがある
* 大魔王バーンと魔王ハドラーが逆になってる

In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
#Jupyterでも実行できるのは知らんかった...
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# PromptHelperとは
* トークンの制約に対応するための汎用プロンプトサポートツール

# LLM（Large Language Model）
* PromptHelperの引数の大部分はLLMに関連する
* つまり最初にLLMが何かを理解する必要がある
* (そもそも入力されたPromptをparseするのはLLMなのでPromptHelperがLLMを引数にとるのは当然である)
* 詳細はここを見よ！
* https://gpt-index.readthedocs.io/en/latest/reference/llm_predictor.html

# そもそも
* LLMPredictorとはLangchainのLLMChainのラッパーであり、LlamdaIndexと統合することができる
* LLMChain ↔ LLMPrecitor ↔ LlamdaIndex
* Our LLMPredictor is a wrapper around Langchain’s LLMChain that allows easy integration into LlamaIndex.
* llmを引数にとり、デフォルトではOpenAI’s text-davinci-003が指定される
* max_input_size: int = 3900, num_output: int = 256
* デフォルトでは入力は3900トークンで出力は256トークン
* つまり長文の回答はできない

# 引数のllm
* OpenAI以外にいも複数のLLMをサポートしている
* OpenAI, Cohere, Hugging Face, etc
* Llama_indexは自身でLLMはサポートしておらず、あくまでもlangchainを介して呼び出しを行う？

In [3]:
from gpt_index import LLMPredictor, GPTSimpleVectorIndex, SimpleDirectoryReader

In [4]:
# OpenAIのAPIのラッパー
from langchain.llms import OpenAI

# 利用例

In [7]:
from llama_index import (
    GPTSimpleVectorIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader,
    GPTTreeIndex
)

from llama_index import (
    GPTKeywordTableIndex, 
    SimpleDirectoryReader, 
    LLMPredictor,
    PromptHelper
)

from gpt_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from langchain import OpenAI

documents = SimpleDirectoryReader('story/dai').load_data()

# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 1024
# set maximum chunk overlap
max_chunk_overlap = 0
# separatorが"/n/n"だとチャンクサイズの計算でエラーになるっぽい。"。"の場合は動く模様。
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, separator="。", chunk_size_limit=256)

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_output))

# build index
index = GPTListIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)
#index = GPTTreeIndex(documents, prompt_helper=prompt_helper)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1
DEBUG:root:> Adding chunk: 序章（デルパ! イルイル! / ダイ爆発!!!）
魔王ハドラー率いる魔王軍と勇者率いる勇者パー...
DEBUG:root:> Adding chunk: 多数のモンスターが住む怪物の島・デルムリン島に赤子の時に漂着し鬼面道士のブラスに育てられた少年...
DEBUG:root:> Adding chunk: 

ダイの旅立ち
ある日、ゴメちゃん以外の島のモンスターたちが凶暴化。辛うじて理性を保っている...
DEBUG:root:> Adding chunk: アバンの弟子となったダイは、魔王を倒し平和を取り戻すべく、アバンの指導を受けて、短期間のうちに...
DEBUG:root:> Adding chunk: 現在のハドラーは「魔界の神」を称する大魔王バーンの力で蘇り、再び地上を制圧するために編成された...
DEBUG:root:> Adding chunk: アバンは弟子たちを護るために自己犠牲呪文・メガンテを使い、その命を散らすもハドラーは生き残り、...
DEBUG:root:> Adding chunk: アバンの遺志を継いだダイは魔王軍を倒すべく、ポップとゴメちゃんと共にデルムリン島から旅立ってい...
DEBUG:root:> Adding chunk: 

魔王軍六大軍団との戦い
ロモス王国に上陸したダイ一行は、同じアバンの弟子である僧侶戦士マァ...
DEBUG:root:> Adding chunk: 続いて訪れたパプニカ王国では魔王軍幹部であると同時にアバンの一番弟子でもある魔剣戦士ヒュンケル...
DEBUG:root:> Adding chunk: アバンへの誤解から魔王軍に加わっていたヒュンケルもまた改心し、ダイ達の仲間となる。一行は氷炎将...
DEBUG:root:> Adding chunk: ハドラーはダイ達「アバンの使徒」を倒そうと躍起になるが、魔王軍内での保身を優先するがあまり、敗...
DEBUG:root:> Adding chunk: そんなダイの前に魔王軍の幹部である竜騎将バランが現れ、ダイの正体が世界の秩序を乱す悪しき存在を...
DEBUG:

In [8]:
summary = index.query("詳しく要約してください", response_mode="tree_summarize")

DEBUG:root:> Searching in chunk: 序章（デルパ! イルイル! / ダイ爆発!!!）
魔王ハドラー率いる魔王軍と勇者率いる勇者パー...
DEBUG:root:> Searching in chunk: 多数のモンスターが住む怪物の島・デルムリン島に赤子の時に漂着し鬼面道士のブラスに育てられた少年...
DEBUG:root:> Searching in chunk: 

ダイの旅立ち
ある日、ゴメちゃん以外の島のモンスターたちが凶暴化。辛うじて理性を保っている...
DEBUG:root:> Searching in chunk: アバンの弟子となったダイは、魔王を倒し平和を取り戻すべく、アバンの指導を受けて、短期間のうちに...
DEBUG:root:> Searching in chunk: 現在のハドラーは「魔界の神」を称する大魔王バーンの力で蘇り、再び地上を制圧するために編成された...
DEBUG:root:> Searching in chunk: アバンは弟子たちを護るために自己犠牲呪文・メガンテを使い、その命を散らすもハドラーは生き残り、...
DEBUG:root:> Searching in chunk: アバンの遺志を継いだダイは魔王軍を倒すべく、ポップとゴメちゃんと共にデルムリン島から旅立ってい...
DEBUG:root:> Searching in chunk: 

魔王軍六大軍団との戦い
ロモス王国に上陸したダイ一行は、同じアバンの弟子である僧侶戦士マァ...
DEBUG:root:> Searching in chunk: 続いて訪れたパプニカ王国では魔王軍幹部であると同時にアバンの一番弟子でもある魔剣戦士ヒュンケル...
DEBUG:root:> Searching in chunk: アバンへの誤解から魔王軍に加わっていたヒュンケルもまた改心し、ダイ達の仲間となる。一行は氷炎将...
DEBUG:root:> Searching in chunk: ハドラーはダイ達「アバンの使徒」を倒そうと躍起になるが、魔王軍内での保身を優先するがあまり、敗...
DEBUG:root:> Searching in chunk: そんなダイの前に魔王軍の幹部である竜騎将バランが現れ、ダイの正体が世界の秩序を乱す悪し

DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openai.com:443
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=22378 request_id=7dadb9fccdb0c132893f3cb8be1464f3 response_code=200
DEBUG:root:
ダイはデルムリン島で育てられた少年で、勇者になることを夢見ていた。ある日、モンスターが凶暴化したことから、魔王ハドラーが復活したことを察する。そこに勇者の家庭教師を名乗る謎の人物アバンとその弟子の魔法使いポップが現れ、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れ、ダイを弟子として修行させる。その間にハドラーが島に来襲し、アバンは自己犠牲呪文・メガンテを使い、その命を散らすもハドラーは生き残る。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。ダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立つ。ダイ一行はロモス王国とパプニカ王国を占拠した魔王軍幹部を打ち破り、レオナの救出にも成功した。ハドラーはダイ達を倒そうと躍起になるが、敗北を続ける。そんな中、ダイの前に魔王軍の幹部である竜騎将バランが現れ、ダイの正体が伝説の存在「竜（ドラゴン）の騎士」であること、さらには自らの息子であることを告げる。しかし、ダイは拒否し、バランは最後の手段としてポップの犠牲がダイの記憶を蘇らせる。バランはダイの奪還を断念し、何処かに姿を消す。
DEB

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=19022 request_id=a50e31e5ef0c31ef405b63f489efa5d9 response_code=200
DEBUG:root:
レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城を破壊し、バーンパレスへの突入を試みるが、魔宮の門によって閉ざされてしまう。そこでバランが現れ、ダイ一行と共闘する。ハドラーがバーンに反逆し、ダイ一行は脱出するが、バランはダイたちを守るために命を落とす。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行する。ダイ一行はレオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。ハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還したアバンであった。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。ダイの勝利でバーンを倒すことに成功し、世界は救われた。
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Context information is below. \\n---------------------\\n\\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306f\\u30d0\\u30fc\\u30f3\\u3092\\u3082\\u5012\\u3057\\u305f\\u30

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=10013 request_id=173acbecdaf8a5fe34732b1c80911ed7 response_code=200
DEBUG:root:

ダイは勇者になることを夢見ていた少年であり、モンスターが凶暴化したことから魔王ハドラーが復活したと察する。そこに謎の人物アバンとその弟子の魔法使いポップが現れ、ダイを弟子として修行させる。島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救うために勇者の家庭教師として名乗りを上げた。レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城を破壊し、バーンパレスへの突入を試みるが、魔宮の門によって閉ざされてしまう。そこでバランが現れ、ダイ一行と共闘することで魔王ハドラーを倒し、島を救うことに成功した。
DEBUG:root:> Refined response: 

ダイは勇者になることを夢見ていた少年であり、モンスターが凶暴化したことから魔王ハドラーが復活したと察する。そこに謎の人物アバンとその弟子の魔法使いポップが現れ、ダイを弟子として修行させる。島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救うために勇者の家庭教師として名乗りを上げた。レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城を破壊し、バーンパレスへの突入を試みるが、魔宮の門によって閉ざされてしまう。そこでバランが現れ、ダイ一行と共闘することで魔王ハドラーを倒し、島を救うことに成功した。
DEBUG:root:> Refine context: ハドラーがバーンに反逆し、ダイ一行は脱出するが、バランはダイたちを守るために命を落とす。バーン...
DEBUG:openai:message='Request to OpenAI A

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=23650 request_id=2304284f7a0ac1d476ead061630b4a90 response_code=200
DEBUG:root:

ダイは勇者になることを夢見ていた少年であり、モンスターが凶暴化したことから魔王ハドラーが復活したと察する。そこに謎の人物アバンとその弟子の魔法使いポップが現れ、ダイを弟子として修行させる。島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救うために勇者の家庭教師として名乗りを上げた。レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城を破壊し、バーンパレスへの突入を試みるが、魔宮の門によって閉ざされてしまう。そこでバランが現れ、ダイ一行と共闘することで魔王ハドラーを倒し、島を救うことに成功した。ハドラーがバーンに反逆し、ダイ一行は脱出するが、バランはダイたちを守るために命を落とす。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行する。キルバーンはダイ一行を全滅させるべく、隠していた黒の核晶を起動させようとしたが、アバンとマァムがキルバーンを倒し、ダイとポップが核晶を空高く運び、土壇場でダイがポップを地上に戻し、一人爆発の閃光の中に消えた。数週間の捜索を経てもダイの姿は見つからず、ロン・ベルクはダイの剣の魔法宝玉が輝き続けていることから、ダイがどこかで生きていることを確信した。ダイは最後まで島を救うために戦い、その行動により魔王ハドラーを倒し、島を救うことに成功した。
DEBUG:root:> Refined response: 

ダイは勇者になることを夢見ていた少年であり、モンスターが凶暴化したことから魔王ハドラーが復活したと察する。そこに謎の人物アバンとその弟子の魔法使いポップが現れ、ダイを弟子として修行させる。島を破邪呪文・マホカトールで覆い、

In [11]:
summary

Response(response='\n\nダイは勇者になることを夢見ていた少年であり、モンスターが凶暴化したことから魔王ハドラーが復活したと察する。そこに謎の人物アバンとその弟子の魔法使いポップが現れ、ダイを弟子として修行させる。島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救うために勇者の家庭教師として名乗りを上げた。レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城を破壊し、バーンパレスへの突入を試みるが、魔宮の門によって閉ざされてしまう。そこでバランが現れ、ダイ一行と共闘することで魔王ハドラーを倒し、島を救うことに成功した。ハドラーがバーンに反逆し、ダイ一行は脱出するが、バランはダイたちを守るために命を落とす。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行する。キルバーンはダイ一行を全滅させるべく、隠していた黒の核晶を起動させようとしたが、アバンとマァムがキルバーンを倒し、ダイとポップが核晶を空高く運び、土壇場でダイがポップを地上に戻し、一人爆発の閃光の中に消えた。数週間の捜索を経てもダイの姿は見つからず、ロン・ベルクはダイの剣の魔法宝玉が輝き続けていることから、ダイがどこかで生きていることを確信した。ダイは最後まで島を救うために戦い、その行動により魔王ハドラーを倒し、島を救うことに成功した。', source_nodes=[SourceNode(source_text='序章（デルパ! イルイル! / ダイ爆発!!!）\n魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わり、魔王の配下であったモンスターも魔王の支配から解き放たれ、世界に平和が訪れた。\n\nそれから十数年後', doc_id='27a2c956-e1dd-49fd-8e14-9cdfe6ee0495', extra_info=None, node_info={'start': 0, 'end': 116}, similarity=None), SourceNode(source_text='多数のモンスターが住む怪物の島・デルムリン島に赤子の時に漂着し鬼面道士のブラスに育てられた少年ダイは、島唯一の人間として、ゴールデンメタルスライムのゴメちゃんを始めとする友達のモンスターたち

In [12]:
summary.source_nodes

[SourceNode(source_text='序章（デルパ! イルイル! / ダイ爆発!!!）\n魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わり、魔王の配下であったモンスターも魔王の支配から解き放たれ、世界に平和が訪れた。\n\nそれから十数年後', doc_id='27a2c956-e1dd-49fd-8e14-9cdfe6ee0495', extra_info=None, node_info={'start': 0, 'end': 116}, similarity=None),
 SourceNode(source_text='多数のモンスターが住む怪物の島・デルムリン島に赤子の時に漂着し鬼面道士のブラスに育てられた少年ダイは、島唯一の人間として、ゴールデンメタルスライムのゴメちゃんを始めとする友達のモンスターたちと共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中でも何度かの冒険や騒動もあり、その過程でダイはパプニカ王国王女のレオナと知り合う', doc_id='27a2c956-e1dd-49fd-8e14-9cdfe6ee0495', extra_info=None, node_info={'start': 117, 'end': 283}, similarity=None),
 SourceNode(source_text='\n\nダイの旅立ち\nある日、ゴメちゃん以外の島のモンスターたちが凶暴化。辛うじて理性を保っているブラスは魔王が復活したことを察する。そこに勇者の家庭教師を名乗る謎の人物アバンとその弟子の魔法使いポップが現れ、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはレオナからダイの事を聞き、その将来性を見込んで島を訪れたのである', doc_id='27a2c956-e1dd-49fd-8e14-9cdfe6ee0495', extra_info=None, node_info={'start': 284, 'end': 459}, similarity=None),
 SourceNode(source_text='アバンの弟子となったダイは、魔王を倒し平和を取り戻すべく、アバンの指導を受けて、短期間のうちに秘められた力を開花させていく。\n\n修行の最